In [10]:
import findspark
findspark.init()
findspark.find
import pyspark
findspark.find()

'C:\\spark\\spark-3.0.0-bin-hadoop2.7'

In [12]:
sc.stop()

In [ ]:
from pyspark import SparkContext
sc=SparkContext('local','local')
rd=sc.parallelize([1,2,3,4])
rd.collect()

In [ ]:
from pyspark.sql import Row
from pyspark.sql import SQLContext
sqlc=SQLContext(sc)
li=[('vish',23,'3455'),('arun',24,'555'),('mohan',34,'666')]
rdd=sc.parallelize(li)
data=rdd.map(lambda x:Row(name=x[0],age=int(x[1]),salary=float(x[2])))
dup=sqlc.createDataFrame(data)
dup.printSchema()
dup.show()

In [13]:
from pyspark import SparkContext
sc=SparkContext()
#18-july-20

In [20]:
from pyspark.sql import SQLContext
url='adult.csv'
from pyspark import SparkFiles
sc.addFile(url)
sqlContext=SQLContext(sc)
df=sqlContext.read.csv(SparkFiles.get(url),header=True,inferSchema=True)
#df.show()
df.select('age','fnlwgt','education').show(10)

+---+------+------------+
|age|fnlwgt|   education|
+---+------+------------+
| 25|226802|        11th|
| 38| 89814|     HS-grad|
| 28|336951|  Assoc-acdm|
| 44|160323|Some-college|
| 18|103497|Some-college|
| 34|198693|        10th|
| 29|227026|     HS-grad|
| 63|104626| Prof-school|
| 24|369667|Some-college|
| 55|104996|     7th-8th|
+---+------+------------+
only showing top 10 rows



In [22]:
from pyspark.sql.types import*
def convertColumn(df,names,newType):
    for name in names:
        df=df.withColumn(name,df[name].cast(newType))
    return df
CONTI_FEATURES=['age','fnlwgt','capital-gain']
#convert_to_string
df_string=convertColumn(df,CONTI_FEATURES,StringType())
#check_dataset
df_string.printSchema()
        

root
 |-- age: string (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: string (nullable = true)
 |-- education: string (nullable = true)
 |-- educational-num: integer (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- capital-gain: string (nullable = true)
 |-- capital-loss: integer (nullable = true)
 |-- hours-per-week: integer (nullable = true)
 |-- native-country: string (nullable = true)
 |-- income: string (nullable = true)



In [26]:
#new_coulmn
from pyspark.ml.feature import StringIndexer
stringIndexer=StringIndexer(inputCol='age',outputCol='newage')
model=stringIndexer.fit(df)
df=model.transform(df)
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- educational-num: integer (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- capital-gain: integer (nullable = true)
 |-- capital-loss: integer (nullable = true)
 |-- hours-per-week: integer (nullable = true)
 |-- native-country: string (nullable = true)
 |-- income: string (nullable = true)
 |-- newage: double (nullable = false)



In [27]:
df.groupby('education').count().sort('count',ascending=False).show()

+------------+-----+
|   education|count|
+------------+-----+
|     HS-grad|15784|
|Some-college|10878|
|   Bachelors| 8025|
|     Masters| 2657|
|   Assoc-voc| 2061|
|        11th| 1812|
|  Assoc-acdm| 1601|
|        10th| 1389|
|     7th-8th|  955|
| Prof-school|  834|
|         9th|  756|
|        12th|  657|
|   Doctorate|  594|
|     5th-6th|  509|
|     1st-4th|  247|
|   Preschool|   83|
+------------+-----+



In [31]:
#df.describe()
df.describe('capital-gain').show()

DataFrame[summary: string, age: string, workclass: string, fnlwgt: string, education: string, educational-num: string, marital-status: string, occupation: string, relationship: string, race: string, gender: string, capital-gain: string, capital-loss: string, hours-per-week: string, native-country: string, income: string, newage: string]

In [33]:
#remove_columns(drop and dropna)
df.drop('education').columns

['age',
 'workclass',
 'fnlwgt',
 'educational-num',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'gender',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'native-country',
 'income',
 'newage']

In [36]:
#people_above_40
#df.filter(df.age >40).count()
df.groupby('marital-status').agg({'capital-gain':'mean'}).show()

+--------------------+------------------+
|      marital-status| avg(capital-gain)|
+--------------------+------------------+
|           Separated| 581.8424836601307|
|       Never-married|  384.382639449029|
|Married-spouse-ab...| 629.0047770700637|
|            Divorced| 793.6755615860094|
|             Widowed| 603.6442687747035|
|   Married-AF-spouse|2971.6216216216217|
|  Married-civ-spouse|1739.7006121810625|
+--------------------+------------------+



In [38]:
#datapreprocessing
from pyspark.sql.functions import*
age_square1=df.select(col("age")**2)
#print(age_square1)
#transform
df=df.withColumn("age_square",col("age")**2)
df.select('age','age_square').show()

+---+----------+
|age|age_square|
+---+----------+
| 25|     625.0|
| 38|    1444.0|
| 28|     784.0|
| 44|    1936.0|
| 18|     324.0|
| 34|    1156.0|
| 29|     841.0|
| 63|    3969.0|
| 24|     576.0|
| 55|    3025.0|
| 65|    4225.0|
| 36|    1296.0|
| 26|     676.0|
| 58|    3364.0|
| 48|    2304.0|
| 43|    1849.0|
| 20|     400.0|
| 43|    1849.0|
| 37|    1369.0|
| 40|    1600.0|
+---+----------+
only showing top 20 rows



In [40]:
COLUMNS=['age','age_square','workclass','fnlwgt','education','marital-status']
df1=df.select(COLUMNS)
df1.show(5)

+---+----------+---------+------+------------+------------------+
|age|age_square|workclass|fnlwgt|   education|    marital-status|
+---+----------+---------+------+------------+------------------+
| 25|     625.0|  Private|226802|        11th|     Never-married|
| 38|    1444.0|  Private| 89814|     HS-grad|Married-civ-spouse|
| 28|     784.0|Local-gov|336951|  Assoc-acdm|Married-civ-spouse|
| 44|    1936.0|  Private|160323|Some-college|Married-civ-spouse|
| 18|     324.0|        ?|103497|Some-college|     Never-married|
+---+----------+---------+------+------------+------------------+
only showing top 5 rows



In [42]:
df.groupby('native-country').agg({'native-country':'count'}).sort(asc('count(native-country)')).show()

+--------------------+---------------------+
|      native-country|count(native-country)|
+--------------------+---------------------+
|  Holand-Netherlands|                    1|
|             Hungary|                   19|
|            Honduras|                   20|
|            Scotland|                   21|
|                Laos|                   23|
|Outlying-US(Guam-...|                   23|
|          Yugoslavia|                   23|
|     Trinadad&Tobago|                   27|
|            Cambodia|                   28|
|            Thailand|                   30|
|                Hong|                   30|
|             Ireland|                   37|
|              France|                   38|
|             Ecuador|                   45|
|                Peru|                   46|
|              Greece|                   49|
|           Nicaragua|                   49|
|                Iran|                   59|
|              Taiwan|                   65|
|         

In [43]:
#filterr(not-holand)
df.filter(df['native-country']!='Holand-Netherlands').show()

+---+----------------+------+------------+---------------+------------------+-----------------+-------------+------------------+------+------------+------------+--------------+--------------+------+------+----------+
|age|       workclass|fnlwgt|   education|educational-num|    marital-status|       occupation| relationship|              race|gender|capital-gain|capital-loss|hours-per-week|native-country|income|newage|age_square|
+---+----------------+------+------------+---------------+------------------+-----------------+-------------+------------------+------+------------+------------+--------------+--------------+------+------+----------+
| 25|         Private|226802|        11th|              7|     Never-married|Machine-op-inspct|    Own-child|             Black|  Male|           0|           0|            40| United-States| <=50K|  16.0|     625.0|
| 38|         Private| 89814|     HS-grad|              9|Married-civ-spouse|  Farming-fishing|      Husband|             White|  Ma

In [ ]:
#count txt_lines
import pyspark 
sc.stop()
sc=pyspark.SparkContext()
txt=sc.textFile('mt.txt')
print(txt.count())